In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
%matplotlib inline

In [2]:
# nc = pd.read_csv('./data/hmda_gancscva.csv')

# Add row indicating individual's unique ID for better tracking
# nc['id'] = range(0, nc.shape[0])
# nc.to_csv('./data/hmda_gancscva_withid.csv')

In [3]:
# If the id csv already exists,
nc = pd.read_csv('./data/hmda_gancscva_withid.csv')

C:\Users\Phoenix\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (30,31,32,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Row Reduction

In [4]:
'''
The focus is on individual data, not institution data. Therefore, remove all rows where 
applicant_sex, applicant_race_name_1, applicant_ethnicity have "na" or "Information not provided by 
applicant in mail, Internet, or telephone application" values.
'''
nc = nc[nc.applicant_sex_name != 'Not applicable'] 
nc = nc[nc.applicant_sex_name != 'Information not provided by applicant in mail, Internet, or telephone application']
nc = nc[nc.applicant_race_name_1 != 'Not applicable']
nc = nc[nc.applicant_race_name_1 != 'Information not provided by applicant in mail, Internet, or telephone application']
nc = nc[nc.applicant_ethnicity_name != 'Not applicable']
nc = nc[nc.applicant_ethnicity_name != 'Information not provided by applicant in mail, Internet, or telephone application']

'''
Remove rows where the loan application is incomplete or withdrawn by applicant. We remove any row with
information about preapproval as we consider preapproval information as not important for our purposes.
'''
nc = nc[nc.action_taken_name != 'Application withdrawn by applicant']
nc = nc[nc.action_taken_name != 'File closed for incompleteness']
nc = nc[nc.action_taken_name != 'Preapproval request approved but not accepted']
nc = nc[nc.action_taken_name != 'Preapproval request denied by financial institution']

'''
Remove all the HOEPA loan applications since there are only 46 (out of 464109 rows).
'''
nc = nc[nc.hoepa_status_name != 'HOEPA loan']

### Column Reduction

In [5]:
nc = nc.drop(['rate_spread', 'state_name', 'state_abbr', 'sequence_number', 'respondent_id', 'purchaser_type_name', 'preapproval_name', 'msamd_name', 'hoepa_status_name', 'edit_status_name', 'denial_reason_name_3', 'denial_reason_name_2', 'denial_reason_name_1', 'county_name', 'co_applicant_race_name_5', 'co_applicant_race_name_4', 'co_applicant_race_name_3', 'co_applicant_race_name_2', 'co_applicant_race_name_1', 'co_applicant_ethnicity_name', 'census_tract_number', 'as_of_year', 'application_date_indicator', 'applicant_race_name_5', 'applicant_race_name_4', 'applicant_race_name_3', 'applicant_race_name_2', 'agency_name'], axis=1)

# Drop the 'Unnamed' column
nc = nc.drop(nc.columns[0], axis=1)
nc.head()

,Unnamed: 0.1,tract_to_msamd_income,population,minority_population,number_of_owner_occupied_units,number_of_1_to_4_family_units,loan_amount_000s,hud_median_family_income,applicant_income_000s,property_type_name,...,loan_type_name,loan_purpose_name,lien_status_name,co_applicant_sex_name,applicant_sex_name,applicant_race_name_1,applicant_ethnicity_name,agency_abbr,action_taken_name,id
0,0,63.500000,7067.0,72.080002,1201.0,1303.0,400.0,107600.0,90.0,One-to-four family dwelling (other than manufa...,...,Conventional,Home purchase,Secured by a first lien,Male,Female,White,Not Hispanic or Latino,HUD,Loan originated,0
4,28,88.040001,4863.0,41.950001,934.0,1585.0,128.0,67600.0,50.0,Manufactured housing,...,Conventional,Home purchase,Secured by a first lien,No co-applicant,Male,White,Not Hispanic or Latino,FDIC,Loan originated,4
5,30,63.500000,7067.0,72.080002,1201.0,1303.0,455.0,107600.0,94.0,One-to-four family dwelling (other than manufa...,...,Conventional,Home purchase,Secured by a first lien,Female,Male,Asian,Not Hispanic or Latino,HUD,Loan originated,5
7,53,61.959999,4523.0,55.209999,1340.0,2219.0,78.0,68800.0,56.0,Manufactured housing,...,Conventional,Home purchase,Secured by a first lien,Female,Male,Black or African American,Not Hispanic or Latino,HUD,Application denied by financial institution,7
8,65,162.149994,4015.0,32.580002,1053.0,1147.0,714.0,107600.0,240.0,One-to-four family dwelling (other than manufa...,...,Conventional,Home purchase,Secured by a first lien,No co-applicant,Male,Black or African American,Not Hispanic or Latino,CFPB,Loan originated,8


### Processing Numerical Features

In [6]:
'''
In order to perform log transformation, remove the rows where the values in columns in numeric_features 
equals to 0.
'''
numeric_features = ['tract_to_msamd_income', 'population', 'minority_population', 'number_of_owner_occupied_units','number_of_1_to_4_family_units','loan_amount_000s','hud_median_family_income', 'applicant_income_000s']
for feature in numeric_features:
    nc = nc[nc[feature] != 0]

    
# Log transformation
log_trans_features = ['tract_to_msamd_income','loan_amount_000s', 'applicant_income_000s']

for feature in log_trans_features:
    nc[feature] = np.log(nc[feature])
    
    
# Square root transformation
sqrt_trans_features = ['population', 'minority_population', 'number_of_owner_occupied_units','number_of_1_to_4_family_units']

for feature in sqrt_trans_features:
    nc[feature] = np.sqrt(nc[feature])
    
    
# MaxMinScaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0, 1))
nc[['tract_to_msamd_income', 'population', 'minority_population', 'number_of_owner_occupied_units','number_of_1_to_4_family_units','loan_amount_000s','hud_median_family_income', 'applicant_income_000s']] = scaler.fit_transform(nc[['tract_to_msamd_income', 'population', 'minority_population', 'number_of_owner_occupied_units','number_of_1_to_4_family_units','loan_amount_000s','hud_median_family_income', 'applicant_income_000s']])

In [7]:
# # Check resulting distribution
# nc[['tract_to_msamd_income', 'population', 'minority_population', 'number_of_owner_occupied_units','number_of_1_to_4_family_units','loan_amount_000s','hud_median_family_income', 'applicant_income_000s']].hist(bins=100, figsize=(15, 15))

In [8]:
nc = nc.dropna()

### Processing Categorical Features

In [9]:
# Binarization
nc['action_taken_name'] = nc['action_taken_name'].apply(lambda x: 0 if x == 'Application denied by financial institution' else 1)
nc['property_type_name'] = nc['property_type_name'].apply(lambda x: '1' if x == 'One-to-four family dwelling (other than manufactured housing)' else '0')
nc['owner_occupancy_name'] = nc['owner_occupancy_name'].apply(lambda x: '1' if x == 'Owner-occupied as a principal dwelling' else '0')
nc['loan_type_name'] = nc['loan_type_name'].apply(lambda x: '1' if x == 'Conventional' else '0')
nc['loan_purpose_name'] = nc['loan_purpose_name'].apply(lambda x: '1' if x == 'Home purchase' else '0')
nc['lien_status_name'] = nc['lien_status_name'].apply(lambda x: '1' if x == 'Secured by a first lien' else '0')
nc['co_applicant_sex_name'] = nc['co_applicant_sex_name'].apply(lambda x: '0' if (x == 'Female' or x == 'Male') else '1') # if there is a co_applicant
# nc['applicant_race_name_1'] = nc['applicant_race_name_1'].apply(lambda x: '1' if x == 'White' else '0')
nc['agency_abbr'] = nc['agency_abbr'].apply(lambda x: '1' if (x == 'HUD' or x == 'CFPB') else '0')

In [10]:
nc.head()

,Unnamed: 0.1,tract_to_msamd_income,population,minority_population,number_of_owner_occupied_units,number_of_1_to_4_family_units,loan_amount_000s,hud_median_family_income,applicant_income_000s,property_type_name,...,loan_type_name,loan_purpose_name,lien_status_name,co_applicant_sex_name,applicant_sex_name,applicant_race_name_1,applicant_ethnicity_name,agency_abbr,action_taken_name,id
0,0,0.611899,0.502730,0.845208,0.405056,0.387398,0.472659,1.000000,0.351716,1,...,1,1,1,0,Female,White,Not Hispanic or Latino,1,1,0
4,28,0.682300,0.399440,0.638842,0.354274,0.433221,0.382771,0.361022,0.305773,0,...,1,1,1,1,Male,White,Not Hispanic or Latino,0,1,4
5,30,0.611899,0.502730,0.845208,0.405056,0.387398,0.482823,1.000000,0.355115,1,...,1,1,1,0,Male,Asian,Not Hispanic or Latino,1,1,5
7,53,0.606609,0.381550,0.736582,0.429251,0.523195,0.343695,0.380192,0.314632,0,...,1,1,1,0,Male,Black or African American,Not Hispanic or Latino,1,0,7
8,65,0.813886,0.353517,0.560011,0.377699,0.359895,0.518369,1.000000,0.428380,1,...,1,1,1,1,Male,Black or African American,Not Hispanic or Latino,1,1,8


### One Hot Encode and Save File

In [11]:
# Set filename
nc = pd.get_dummies(nc)
# Getting rid of spaces and other annoying things in colnames
nc.columns = nc.columns.str.replace(' ', '_')
nc.columns = nc.columns.str.replace('-', '_')
nc.columns = nc.columns.str.replace('(', '_')
nc.columns = nc.columns.str.replace(')', '_')
nc.columns = nc.columns.str.replace('/', '_')

In [12]:
nc.head()

,Unnamed:_0.1,tract_to_msamd_income,population,minority_population,number_of_owner_occupied_units,number_of_1_to_4_family_units,loan_amount_000s,hud_median_family_income,applicant_income_000s,action_taken_name,...,applicant_sex_name_Male,applicant_race_name_1_American_Indian_or_Alaska_Native,applicant_race_name_1_Asian,applicant_race_name_1_Black_or_African_American,applicant_race_name_1_Native_Hawaiian_or_Other_Pacific_Islander,applicant_race_name_1_White,applicant_ethnicity_name_Hispanic_or_Latino,applicant_ethnicity_name_Not_Hispanic_or_Latino,agency_abbr_0,agency_abbr_1
0,0,0.611899,0.502730,0.845208,0.405056,0.387398,0.472659,1.000000,0.351716,1,...,0,0,0,0,0,1,0,1,0,1
4,28,0.682300,0.399440,0.638842,0.354274,0.433221,0.382771,0.361022,0.305773,1,...,1,0,0,0,0,1,0,1,1,0
5,30,0.611899,0.502730,0.845208,0.405056,0.387398,0.482823,1.000000,0.355115,1,...,1,0,1,0,0,0,0,1,0,1
7,53,0.606609,0.381550,0.736582,0.429251,0.523195,0.343695,0.380192,0.314632,0,...,1,0,0,1,0,0,0,1,0,1
8,65,0.813886,0.353517,0.560011,0.377699,0.359895,0.518369,1.000000,0.428380,1,...,1,0,0,1,0,0,0,1,0,1


In [13]:
out_file = "hmda_gancscva_clean_allraces.csv"
nc.to_csv(out_file)

# Create balanced datasets

In [14]:
import random

random.seed(2019)

In [15]:
# col1 and val1 correspond to the values we want to exist in 1/2 of the data
# col2 and val2 correspond to the values we want to exist in the other half
def balance(data, col1, col2, val1, val2):
    first_half = data[data[col1] == val1]
    second_half = data[data[col2] == val2]
    smaller_size = min(first_half.shape[0], second_half.shape[0]) # want to down sample to the smaller set
    
    balanced = first_half.sample(smaller_size)
    balanced = balanced.append(second_half.sample(smaller_size))
    return(balanced)

In [16]:
accpt_deny = balance(nc, "action_taken_name", "action_taken_name", 0, 1)
black_white = balance(nc, "applicant_race_name_1_Black_or_African_American", "applicant_race_name_1_White", 1, 1)
hisp_nothisp = balance(nc, "applicant_ethnicity_name_Hispanic_or_Latino", "applicant_ethnicity_name_Hispanic_or_Latino", 1, 0)
female_male = balance(nc, "applicant_sex_name_Female", "applicant_sex_name_Female", 1, 0)

In [18]:
# Need to filter irrelevant columns (like other races in black_white)
black_white[~((black_white.applicant_race_name_1_Black_or_African_American == 0) & 
            (black_white.applicant_race_name_1_White == 0))]
black_white = black_white.drop(['applicant_race_name_1_American_Indian_or_Alaska_Native', 'applicant_race_name_1_Asian', 'applicant_race_name_1_Native_Hawaiian_or_Other_Pacific_Islander'], axis=1)

In [19]:
black_white.head()

,Unnamed:_0.1,tract_to_msamd_income,population,minority_population,number_of_owner_occupied_units,number_of_1_to_4_family_units,loan_amount_000s,hud_median_family_income,applicant_income_000s,action_taken_name,...,co_applicant_sex_name_0,co_applicant_sex_name_1,applicant_sex_name_Female,applicant_sex_name_Male,applicant_race_name_1_Black_or_African_American,applicant_race_name_1_White,applicant_ethnicity_name_Hispanic_or_Latino,applicant_ethnicity_name_Not_Hispanic_or_Latino,agency_abbr_0,agency_abbr_1
239239,5199298,0.733892,1.000000,0.625249,0.890649,0.947343,0.449701,0.386581,0.370876,1,...,0,1,1,0,1,0,0,1,0,1
501318,7724003,0.755343,0.510821,0.834394,0.512155,0.567585,0.308615,0.447284,0.295782,1,...,0,1,1,0,1,0,0,1,0,1
273141,5786320,0.738560,0.448423,0.584464,0.519271,0.515152,0.393689,0.538339,0.297538,0,...,0,1,1,0,1,0,0,1,0,1
914572,10276334,0.678821,0.507768,0.836714,0.479500,0.512891,0.236330,0.329073,0.359166,1,...,1,0,0,1,1,0,0,1,1,0
1529940,13578786,0.735638,0.448470,0.802377,0.446705,0.545294,0.402322,0.014377,0.334275,0,...,0,1,1,0,1,0,0,1,0,1


In [20]:
out_file = "./data/hmda_gancscva_clean_balance_action.csv"
accpt_deny.to_csv(out_file)
out_file = "./data/hmda_gancscva_clean_balance_race.csv"
black_white.to_csv(out_file)
out_file = "./data/hmda_gancscva_clean_balance_ethni.csv"
hisp_nothisp.to_csv(out_file)
out_file = "./data/hmda_gancscva_clean_balance_sex.csv"
female_male.to_csv(out_file)

In [21]:
# Balance by action taken and race
prot_accpt = nc[nc["applicant_race_name_1_Black_or_African_American"] == 1]
prot_accpt = prot_accpt[prot_accpt["action_taken_name"] == 1]

prot_deny = nc[nc["applicant_race_name_1_Black_or_African_American"] == 1]
prot_deny = prot_deny[prot_deny["action_taken_name"] == 0]

nprot_accpt = nc[nc["applicant_race_name_1_White"] == 1]
nprot_accpt = nprot_accpt[nprot_accpt["action_taken_name"] == 1]

nprot_deny = nc[nc["applicant_race_name_1_White"] == 1]
nprot_deny = nprot_deny[nprot_deny["action_taken_name"] == 0]

In [22]:
smaller_size = min(prot_accpt.shape[0], prot_deny.shape[0], nprot_accpt.shape[0], nprot_deny.shape[0])

balanced = prot_accpt.sample(smaller_size)
balanced = balanced.append(prot_deny.sample(smaller_size))
balanced = balanced.append(nprot_accpt.sample(smaller_size))
balanced = balanced.append(nprot_deny.sample(smaller_size))

In [23]:
balanced = balanced.drop(['Unnamed:_0.1', 'applicant_race_name_1_American_Indian_or_Alaska_Native', 'applicant_race_name_1_Asian',
              'applicant_race_name_1_Native_Hawaiian_or_Other_Pacific_Islander'], axis=1)

In [24]:
out_file = "./data/hmda_gancscva_clean_balance_action_race.csv"
balanced.to_csv(out_file)